# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,panacan,9.2463,118.4082,29.93,71,91,6.03,PH,1722912888
1,1,villa zaragoza,22.0333,-100.7333,16.86,75,0,2.40,MX,1722912890
2,2,albany,42.6001,-73.9662,23.29,56,100,2.81,US,1722912552
3,3,thompson,55.7435,-97.8558,19.09,48,75,1.54,CA,1722912893
4,4,letterkenny,54.9500,-7.7333,7.91,98,9,1.66,IE,1722912894


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=650,
    frame_height=500,
    size="Humidity",
    color="City"
    )
    
# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cond_df = city_data_df.loc[((city_data_df["Max Temp"] <= 27.0) & (city_data_df["Max Temp"] >= 21.0))
                                 & (city_data_df["Wind Speed"] <= 4.5),:]

# Drop any rows with null values
ideal_cond_df = ideal_cond_df.dropna(how="any")

# Display sample data
ideal_cond_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,albany,42.6001,-73.9662,23.29,56,100,2.81,US,1722912552
10,10,bethel,41.3712,-73.4140,25.36,87,99,1.22,US,1722912712
34,34,shohimardon,39.9839,71.8028,22.43,34,0,2.00,UZ,1722912930
43,43,nikolayevsk-on-amure,53.1406,140.7300,21.05,81,100,1.52,RU,1722912940
48,48,kailua-kona,19.6406,-155.9956,25.71,96,100,2.94,US,1722912788


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cond_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

C:\Users\rahul\AppData\Local\Temp\ipykernel_1416\4062331021.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
2,albany,US,42.6001,-73.9662,56,
10,bethel,US,41.3712,-73.4140,87,
34,shohimardon,UZ,39.9839,71.8028,34,
43,nikolayevsk-on-amure,RU,53.1406,140.7300,81,
48,kailua-kona,US,19.6406,-155.9956,96,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "limit":10,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
shohimardon - nearest hotel: Муминжон ота гостиница
nikolayevsk-on-amure - nearest hotel: Север
kailua-kona - nearest hotel: Kona Seaside Hotel
port mathurin - nearest hotel: Escale Vacances
tura - nearest hotel: Hotel Polo Orchid
porto novo - nearest hotel: Hôtel 6500 CFA
breves - nearest hotel: No hotel found
sig - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
sidrolandia - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
wenshui - nearest hotel: TianWan Hotel
arenapolis - nearest hotel: No hotel found
bidur - nearest hotel: Loong Fatt Hotel
'ain deheb - nearest hotel: No hotel found
axim - nearest hotel: Axim Beach Hotel
elizabeth city - nearest hotel: Quality Inn
cambridge - nearest hotel: Travelodge
singtam - nearest hotel: Hotel Anand
chaihe - nearest hotel: No hotel found
whistler - nearest hotel: WorldMark Whistler 

,City,Country,Lat,Lng,Humidity,Hotel Name
2,albany,US,42.6001,-73.9662,56,No hotel found
10,bethel,US,41.3712,-73.4140,87,Hampton Inn Danbury
34,shohimardon,UZ,39.9839,71.8028,34,Муминжон ота гостиница
43,nikolayevsk-on-amure,RU,53.1406,140.7300,81,Север
48,kailua-kona,US,19.6406,-155.9956,96,Kona Seaside Hotel
...,...,...,...,...,...,...
535,tame,CO,6.4610,-71.7300,89,hotel libertadores
538,broome,US,42.2506,-75.8330,88,No hotel found
542,puerto tirol,AR,-27.3722,-59.0821,96,No hotel found
544,kievka,KZ,50.2619,71.5514,45,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=650,
    frame_height=500,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
    )

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)